In [7]:
# convert color for colorblind people 
# from http://mkweb.bcgsc.ca/colorblind/math.mhtml#page-container

# HOW TO USE: Copy an image to clipboard, then run this cell. A window opens
# that shows original, deuteranopia, tritanopia and black & white images.
# Press ESC to close the window

import numpy as np
import cv2
from PIL import ImageGrab

# Tprotanopia = np.array( [[ 0.170557, 0.829443, 0], 
#                          [ 0.170557, 0.829443, 0], 
#                          [-0.00452, 0.00452, 1]] )
# Tdeuteranopia = np.array( [[ 0.33066, 0.66934, 0], 
#                            [ 0.33066, 0.66934, 0], 
#                            [-0.02786, 0.02786, 1]] )
# Ttritanopia = np.array( [[1, 0.1274,-0.1274], 
#                          [0, 0.8739, 0.1261], 
#                          [0, 0.8739, 0.1261]] )
# Tachromatopsia = np.array( [[0.2123, 0.7152, 0.0722], 
#                             [0.2123, 0.7152, 0.0722], 
#                             [0.2123, 0.7152, 0.0722]] )

def ColorblindSRGB(SRGB, anomaly=1):
    Tanomaly = np.array( [ # choose 1 of the 4 2D arrays from this 3D array
        [[0.2123,0.7152,0.0722], [0.2123,0.7152,0.0722],[0.2123,0.7152,0.0722]] 
        , [[0.170557,0.829443,0], [0.170557,0.829443,0], [-0.00452,0.00452,1]] 
        , [[0.33066,0.669339,0], [0.33066,0.669339,0], [-0.02786,0.02786,1]] 
        , [[1,0.12739,-0.12739], [0,0.87391,0.12609], [0,0.87391,0.12609]] 
        ] ) # [0=Achromatopsia 1=Protanopia 2=Deuteranopia 3=Tritanopia]
    SRGB = cv2.cvtColor(SRGB, cv2.COLOR_BGR2RGB)
    Ishape = np.shape(SRGB)[:2]
#     print(Tanomaly[anomaly, :, :], Ishape)
    LRGB = np.where( SRGB/255<=0.04045, SRGB/255/12.92, \
                     (( SRGB/255+0.055)/1.055 )**2.4)
    AA = np.copy(LRGB)#, dtype=np.float32)
    LRGB = np.ravel(AA).reshape((-1,3))
    newLRGB = np.dot(LRGB,Tanomaly[anomaly, :, :].T). reshape( \
                    (Ishape[0],Ishape[1],3) )  # newLRGB = np.dot(Tmat,LRGB)

    newLRGB = np.where(newLRGB<0, 0, newLRGB) # clip <0 to 0
    newLRGB = np.where(newLRGB>1, 1, newLRGB) # clip >1 to 1
    newSRGB = np.where( newLRGB<=0.0031308, newLRGB*255*12.92, \
                        255*( 1.055*(newLRGB**(1/2.4)) - 0.055 ) )
    print(Ishape, np.shape(newLRGB))
    newSRGB = np.round(newSRGB).astype('uint8')
    return cv2.cvtColor(newSRGB, cv2.COLOR_RGB2BGR)


# IM = np.ones([50,100], dtype=np.uint8)
# srgb=cv2.cvtColor( cv2.merge([153*IM,142*IM,195*IM]) , cv2.COLOR_BGR2RGB)
# srgb = cv2.imread("./untitled2.png",) # 
# srgb = cv2.imread("./rainbow_test3.jpg",) # ("./sample16.jpg",) # ("./rainbow_test2.jpg",) # 

# paste from clipboard
img = ImageGrab.grabclipboard().convert('RGB'); 
srgb = cv2.cvtColor( np.array(img) , cv2.COLOR_RGB2BGR)

if((resize:=1)==1): # do resize:=1 for resizing image
    imW = np.shape(srgb)[:2]; scale = max(1,np.ceil((imW[0]-64)/384))
    size = ( int(imW[1]/scale), int(imW[0]/scale) )
    srgb = cv2.resize(srgb, size, interpolation = cv2.INTER_NEAREST)

def TxtIm(text, img):
    return cv2.putText(img,text,(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.3, \
                       (191,0,191), 1, cv2.LINE_AA)

# cv2.imshow("Normal", srgb)    
# [0=Achromatopsia 1=Protanopia 2=Deuteranopia 3=Tritanopia]
# ClrBld = ColorblindSRGB(srgb);  cv2.imshow("Default option Protanopia", ClrBld)
Cachr = ColorblindSRGB(srgb, 0);  # cv2.imshow("Achromatopsia", Cachr)
# Cprot = ColorblindSRGB(srgb, 1);  cv2.imshow("Protanopia", Cprot)
Cdeut = ColorblindSRGB(srgb, 2);  # cv2.imshow("Deuteranopia", Cdeut)
Ctrit = ColorblindSRGB(srgb, 3);  # cv2.imshow("Tritanopia", Ctrit)
cv2.imshow(" ", cv2.hconcat([ cv2.vconcat([
                            TxtIm("Normal",srgb),
                            TxtIm("Achromatopsia",Cachr), ]), cv2.vconcat([
                            TxtIm("Deuteranopia (Protanopia is similar)",Cdeut),
                            TxtIm("Tritanopia",Ctrit)]) ]))
cv2.waitKey(0)
cv2.destroyAllWindows()

(412, 598) (412, 598, 3)
(412, 598) (412, 598, 3)
(412, 598) (412, 598, 3)
